Cell 1: Import

In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent  # assuming notebooks/ is one level below root
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

%load_ext autoreload
%autoreload 2

import pandas as pd

from ndw.traffic_speed import parse_trafficspeed

Cell 2: Load traffic speed

In [2]:
df_speed = parse_trafficspeed()
print("Rows:", len(df_speed))
df_speed = df_speed[df_speed["avg_speed_kmh"] != -1]
df_speed.head()

Rows: 19893


,site_id,measurement_time,avg_speed_kmh,flow_veh_per_hour
6,PZH01_MST_0635_01_01,2025-12-02T17:26:00Z,67.0,300.0
9,PZH01_MST_0635_01_00,2025-12-02T17:26:00Z,69.0,600.0
10,PZH01_MST_0982_00,2025-12-02T17:26:00Z,95.0,660.0
76,PZH01_MST_0981_01,2025-12-02T17:26:00Z,70.0,900.0
146,PZH01_MST_0637_01,2025-12-02T17:26:00Z,48.0,840.0


Cell 3: Basic info

In [3]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15148 entries, 6 to 19886
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   site_id            15148 non-null  object 
 1   measurement_time   15148 non-null  object 
 2   avg_speed_kmh      15147 non-null  float64
 3   flow_veh_per_hour  15148 non-null  float64
dtypes: float64(2), object(2)
memory usage: 591.7+ KB


Cell 4: Quick sanity checks

In [4]:
print("Unique sites:", df_speed["site_id"].nunique())
print("measurement_time range:",
      df_speed["measurement_time"].min(),
      "→",
      df_speed["measurement_time"].max())

print("\nSpeed stats (km/h):")
print(df_speed["avg_speed_kmh"].describe())

print("\nFlow stats (veh/h):")
print(df_speed["flow_veh_per_hour"].describe())

Unique sites: 15148
measurement_time range: 2025-12-02T17:25:00Z → 2025-12-02T17:26:00Z

Speed stats (km/h):
count    15147.000000
mean        78.067604
std         46.062008
min          0.000000
25%         52.500000
50%         99.000000
75%        111.000000
max        169.000000
Name: avg_speed_kmh, dtype: float64

Flow stats (veh/h):
count    15148.000000
mean       514.116517
std        507.031714
min          0.000000
25%         60.000000
50%        420.000000
75%        780.000000
max       3420.000000
Name: flow_veh_per_hour, dtype: float64


Cell 5 – Example: top slowest sites right now

In [5]:
# drop rows with missing speed
slow = (
    df_speed.dropna(subset=["avg_speed_kmh"])
    .sort_values("avg_speed_kmh")
    .head(20)
)

slow[["site_id", "measurement_time", "avg_speed_kmh", "flow_veh_per_hour"]]

,site_id,measurement_time,avg_speed_kmh,flow_veh_per_hour
19886,RDH01_TI152R,2025-12-02T17:26:00Z,0.0,0.0
4394,PLB02_271260_RP,2025-12-02T17:26:00Z,0.0,0.0
4395,PLB02_297140_LP,2025-12-02T17:26:00Z,0.0,0.0
19754,GRT02_MORO_4013_2,2025-12-02T17:26:00Z,0.0,0.0
4397,PLB02_295150_RP,2025-12-02T17:26:00Z,0.0,0.0
4398,PLB02_275470_LP,2025-12-02T17:26:00Z,0.0,0.0
4399,PLB02_MSTR013_RP,2025-12-02T17:26:00Z,0.0,0.0
4400,PLB02_572150_RP,2025-12-02T17:26:00Z,0.0,0.0
4401,PLB02_294150_RP,2025-12-02T17:26:00Z,0.0,0.0
4402,PLB02_280550_RP,2025-12-02T17:26:00Z,0.0,0.0


Shapefile

In [6]:
from ndw.measurement_sites import parse_location_raw, load_measurement_sites

df_sites = load_measurement_sites()
print("Measurement sites:", len(df_sites))
df_sites.head()

df_sites_parsed = df_sites.copy()

df_sites_parsed[
    ["lat", "lon", "carriageway_type", "carriageway", "direction_ref"]
] = df_sites_parsed["location_raw"].apply(parse_location_raw)

df_sites_parsed.head()

Measurement sites: 101529


,site_id,version,site_name,location_raw,lat,lon,carriageway_type,carriageway,direction_ref
0,PZH01_MST_0629_00,3,None,52.0263 | 4.634289 | mainCarriageway | 8 | 6.1...,52.026300,4.634289,mainCarriageway,A,positive
1,PZH01_MST_0629_01,3,None,52.0262451 | 4.634219 | mainCarriageway | 8 | ...,52.026245,4.634219,mainCarriageway,A,negative
2,PZH01_MST_0634_02,3,None,51.9836769 | 4.220052 | mainCarriageway | 8 | ...,51.983677,4.220052,mainCarriageway,A,negative
3,PZH01_MST_0635_01_00,4,None,51.994175 | 4.259996 | mainCarriageway | 8 | 6...,51.994175,4.259996,mainCarriageway,A,positive
4,PZH01_MST_0635_01_01,3,None,51.9942322 | 4.259998 | mainCarriageway | 8 | ...,51.994232,4.259998,mainCarriageway,A,negative


In [7]:
df_speed_enriched = df_speed.merge(
    df_sites_parsed[
        ["site_id", "lat", "lon", "carriageway_type", "carriageway", "direction_ref"]
    ],
    on="site_id",
    how="left",
)

df_speed_enriched.head()

,site_id,measurement_time,avg_speed_kmh,flow_veh_per_hour,lat,lon,carriageway_type,carriageway,direction_ref
0,PZH01_MST_0635_01_01,2025-12-02T17:26:00Z,67.0,300.0,51.994232,4.259998,mainCarriageway,A,negative
1,PZH01_MST_0635_01_00,2025-12-02T17:26:00Z,69.0,600.0,51.994175,4.259996,mainCarriageway,A,positive
2,PZH01_MST_0982_00,2025-12-02T17:26:00Z,95.0,660.0,52.127636,4.470652,mainCarriageway,A,positive
3,PZH01_MST_0981_01,2025-12-02T17:26:00Z,70.0,900.0,52.010380,4.456722,mainCarriageway,A,negative
4,PZH01_MST_0637_01,2025-12-02T17:26:00Z,48.0,840.0,52.029686,4.632839,mainCarriageway,A,positive


Save data

In [8]:
output_dir = project_root / "data"
output_dir.mkdir(exist_ok=True)

csv_path = output_dir / "ndw_trafficspeed_enriched.csv"
df_speed_enriched.to_csv(csv_path, index=False)

print("Saved:", csv_path)

Saved: /Users/Bruno/Library/CloudStorage/OneDrive-TUEindhoven/IGNITE/data/data/ndw_trafficspeed_enriched.csv


Further Shapefile to make easier locating

In [9]:
import geopandas as gpd
from shapely.geometry import Point

# Only keep rows with valid coordinates
df_sites_geo = df_sites_parsed.dropna(subset=["lat", "lon"]).copy()

gdf_sites = gpd.GeoDataFrame(
    df_sites_geo,
    geometry=gpd.points_from_xy(df_sites_geo["lon"], df_sites_geo["lat"]),
    crs="EPSG:4326",   # WGS84
)

In [10]:
from ndw.ndw_shapefile_utils import load_shapefile_from_url
gdf_msi = load_shapefile_from_url() 

gdf_sites_rd = gdf_sites.to_crs(28992)
gdf_msi_rd   = gdf_msi.to_crs(28992)

In [11]:
gdf_join = gpd.sjoin_nearest(
    gdf_sites_rd,
    gdf_msi_rd[["road", "carriagew0", "lane", "km", "wegvak", "wegbeheer0", "geometry"]],
    how="left",
    distance_col="dist_m",
)

Final version

In [12]:
df_site_msi = gdf_join[
    ["site_id", "road", "carriagew0", "lane", "km", "wegvak", "wegbeheer0"]
].copy()

df_speed_mega_enriched = df_speed_enriched.merge(
    df_site_msi,
    on="site_id",
    how="left",
)
df_speed_mega_enriched.head()

,site_id,measurement_time,avg_speed_kmh,flow_veh_per_hour,lat,lon,carriageway_type,carriageway,direction_ref,road,carriagew0,lane,km,wegvak,wegbeheer0
0,PZH01_MST_0635_01_01,2025-12-02T17:26:00Z,67.0,300.0,51.994232,4.259998,mainCarriageway,A,negative,A4,R,1,54.487,162293009,RWS West-Nederland Zuid
1,PZH01_MST_0635_01_01,2025-12-02T17:26:00Z,67.0,300.0,51.994232,4.259998,mainCarriageway,A,negative,A4,R,3,54.487,162293009,RWS West-Nederland Zuid
2,PZH01_MST_0635_01_01,2025-12-02T17:26:00Z,67.0,300.0,51.994232,4.259998,mainCarriageway,A,negative,A4,R,2,54.487,162293009,RWS West-Nederland Zuid
3,PZH01_MST_0635_01_00,2025-12-02T17:26:00Z,69.0,600.0,51.994175,4.259996,mainCarriageway,A,positive,A4,R,1,54.487,162293009,RWS West-Nederland Zuid
4,PZH01_MST_0635_01_00,2025-12-02T17:26:00Z,69.0,600.0,51.994175,4.259996,mainCarriageway,A,positive,A4,R,3,54.487,162293009,RWS West-Nederland Zuid


Sort by fasters avg speed or slowest

In [13]:
# sort by avg speed (fastest or slowest)
def sort_by_speed(df=df_speed_mega_enriched, n=20, fastest=True, dropna=True, cols=None):
    """
    Return top n rows sorted by avg_speed_kmh.
    - fastest=True  -> highest speeds first
    - fastest=False -> lowest speeds first
    """
    order = False if fastest else True
    q = df
    if dropna:
        q = q.dropna(subset=["avg_speed_kmh"])
    res = q.sort_values("avg_speed_kmh", ascending=order).head(n)
    if cols is None:
        cols = [
            "site_id",
            "measurement_time",
            "avg_speed_kmh",
            "flow_veh_per_hour",
            "road",
            "carriagew0",
            "lane",
            "km",
        ]
    return res.loc[:, [c for c in cols if c in res.columns]]

# Examples
print("Top 10 fastest sites:")
display(sort_by_speed(n=10, fastest=True))

print("\nTop 10 slowest sites:")
display(sort_by_speed(n=10, fastest=False))

Top 10 fastest sites:


,site_id,measurement_time,avg_speed_kmh,flow_veh_per_hour,road,carriagew0,lane,km
14452,RWS01_MONIBAS_0241hrl0040ra,2025-12-02T17:25:00Z,169.0,60.0,A24,L,2,3.930
14451,RWS01_MONIBAS_0241hrl0040ra,2025-12-02T17:25:00Z,169.0,60.0,A24,L,3,3.930
14450,RWS01_MONIBAS_0241hrl0040ra,2025-12-02T17:25:00Z,169.0,60.0,A24,L,1,3.930
29342,RWS01_MONIBAS_0091hrr0081ra,2025-12-02T17:25:00Z,167.0,660.0,A9,R,2,8.140
29341,RWS01_MONIBAS_0091hrr0081ra,2025-12-02T17:25:00Z,167.0,660.0,A9,R,3,8.140
29340,RWS01_MONIBAS_0091hrr0081ra,2025-12-02T17:25:00Z,167.0,660.0,A9,R,1,8.140
23565,RWS01_MONIBAS_0011hrl1283ra,2025-12-02T17:25:00Z,158.0,120.0,A1,L,2,128.335
23564,RWS01_MONIBAS_0011hrl1283ra,2025-12-02T17:25:00Z,158.0,120.0,A1,L,3,128.335
23563,RWS01_MONIBAS_0011hrl1283ra,2025-12-02T17:25:00Z,158.0,120.0,A1,L,1,128.335
12921,RWS01_MONIBAS_0441hrl0204ra,2025-12-02T17:25:00Z,157.0,480.0,A44,L,1,20.500



Top 10 slowest sites:


,site_id,measurement_time,avg_speed_kmh,flow_veh_per_hour,road,carriagew0,lane,km
42213,RDH01_TI152R,2025-12-02T17:26:00Z,0.0,0.0,A44,L,2,21.100
5943,PLB02_562145_RP,2025-12-02T17:26:00Z,0.0,0.0,A73,L,2,27.717
5942,PLB02_SG005,2025-12-02T17:26:00Z,0.0,0.0,A2,R,3,232.450
5941,PLB02_SG005,2025-12-02T17:26:00Z,0.0,0.0,A2,R,1,232.450
5940,PLB02_SG005,2025-12-02T17:26:00Z,0.0,0.0,A2,R,2,232.450
5939,PLB02_277350_RP,2025-12-02T17:26:00Z,0.0,0.0,A73,d,1,28.200
5938,PLB02_270270_RP,2025-12-02T17:26:00Z,0.0,0.0,A67,R,1,38.175
5937,PLB02_270270_RP,2025-12-02T17:26:00Z,0.0,0.0,A67,R,2,38.175
5936,PLB02_SG006,2025-12-02T17:26:00Z,0.0,0.0,A2,R,3,232.450
5935,PLB02_SG006,2025-12-02T17:26:00Z,0.0,0.0,A2,R,1,232.450
